In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy import stats
import time

### Heston model Monte Carlo simulation

The parameters of the Heston model will be:
- Initial price of the stock/$S_0$: 250
- Initial volatility/$sigma_0$: 0.04
- r: 0
- k: 1
- $theta$: 0.09
- Volvol/$alpha$: 0.3
- p: -0.5
- K: 270
- T: 1 year, with 1000 steps 


In [2]:
L=50 #simulated options
M=50#prices per option

#time
T=1
n=1000


#parameters 
initial_price=250
initial_vol=0.3
r=0.01
k=1
theta=0.09
volvol=0.3
p=-0.5
K=270


### Accrued Monte Carlo

Asset S following the Heston model

$dv_t = k(\theta - v_t)dt + \alpha \sqrt{v_t} dZ_t $

$dW_t= pdZ_t + \sqrt{1-p^2}dB_t$

$dS_t = r S_tdt + \sqrt v_t S_t dW_t  $

In [3]:
#initial volatility, volvol, initial_price, free rate, correlation
def Heston_accrued_Euler(initial_price, initial_vol, r, k, theta, volvol, T, p):
    T=T
    n=1000
    dt=T/n
    
    time=[0]*n
    v=[0]*n
    v[0]=initial_vol
    Zt=[0]*n
    dZ=[0]*n
    Bt=[0]*n
    S=[0]*n
    S[0]=initial_price
    
    for i in range(1,n):
        #update time
        time[i]=time[i-1]+dt
        
        #generate volatility
        Zt[i]=Zt[i-1]+np.random.normal(0,np.sqrt(dt))
        dZ[i-1]=Zt[i]-Zt[i-1]
        
        vol_drift=k*(theta-v[i-1])
        dv=vol_drift*dt + (volvol*np.sqrt(v[i-1])*(dZ[i-1]))
        v[i]=v[i-1]+dv  
        
        #Euler scheme
        if v[i]<0:
            v[i]=0
    
        #generate the correlated brownian dW
        Bt[i]=Bt[i-1]+np.random.normal(0,np.sqrt(dt))
        dB=Bt[i]-Bt[i-1]
        
        #generate stock price
        dS=r*S[i-1]*dt + (np.sqrt(v[i-1])*S[i-1]*(p*dZ[i-1] + np.sqrt(1-p**2)*dB))
        S[i]=S[i-1]+dS
                       
    return S

def Heston_accrued_Milstein(initial_price, initial_vol, r, k, theta, volvol, T, p):
    T=T
    n=1000
    dt=T/n
    
    time=[0]*n
    v=[0]*n
    v[0]=initial_vol
    Zt=[0]*n
    dZ=[0]*n
    Bt=[0]*n
    S=[0]*n
    S[0]=initial_price
    
    for i in range(1,n):
        #update time
        time[i]=time[i-1]+dt
        
        #generate volatility
        Zt[i]=Zt[i-1]+np.random.normal(0,np.sqrt(dt))
        dZ[i-1]=Zt[i]-Zt[i-1]
        
        vol_drift=k*(theta-v[i-1])
        #computing dv according to Milschein scheme
        dv=vol_drift*dt + (volvol*np.sqrt(v[i-1])*dZ[i-1]) + ((volvol*volvol)/4)*((dZ[i-1]**2)-dt)
        v[i]=v[i-1]+dv 
    
        #generate the correlated brownian dW
        Bt[i]=Bt[i-1]+np.random.normal(0,np.sqrt(dt))
        dB=Bt[i]-Bt[i-1]
        
        #generate stock price
        dS=r*S[i-1]*dt + (np.sqrt(v[i-1])*S[i-1]*(p*dZ[i-1] + np.sqrt(1-p**2)*dB))
        S[i]=S[i-1]+dS
                       
    return S

In [19]:
#simulation for M, L times
payoff=np.zeros(M)
last_price=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #compute M paths
        last_price[i]=Heston_accrued_Milstein(initial_price, initial_vol,r, k, theta, volvol ,T, p)[-1]
        
    #computing payoff
    payoff=last_price-K
    payoff[payoff<0]=0
    mean_pay[x]=np.exp(-r*T)*np.mean(payoff)

end = time.time()

print(end - start)

23.511786222457886


Mean and variance of the option payoff

In [20]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

39.2714329737421
9.859765994292058


### Control variate Method

$\theta_c = Y + c (Z - E(Z))$

Where 

$Y= h(S_T)=max(S_T-K,0)$ 

$Z= S_T $

$E(Z)= S_0 exp(rT)=S_0$

$c=Cov(h(S_T),S_T)/Var(S_t)$

In [21]:
payoff=np.zeros(M)
last_price=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #compute M paths
            last_price[i]=Heston_accrued_Milstein(initial_price, initial_vol,r, k, theta, volvol ,T, p)[-1]

    #computing payoff
    payoff=last_price-K
    payoff[payoff<0]=0
    
    #control variate
    Y=np.exp(-r*T)*payoff
    Z=last_price #our control variate
    c=-np.cov(Y,Z)[0][1]/np.var(Z)
    control_variate=Y+c*(Z-(initial_price*np.exp(r*T)))
    
    mean_pay[x]=np.mean(control_variate)

end = time.time()

print(end-start)

23.18173909187317


In [22]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

37.70387553012093
5.804881084546249


### Antithetic Variates Method

$\hat{\theta} = (Y + Y2)/2$

Where 

$Y= h(S_T)$ 

$Y2= h(S_Tneg)$ 


In [8]:
def Heston_antithetic(initial_price, initial_vol,r, k, theta, volvol ,T, p):
    n=1000
    dt=T/n
    
    time=[0]*n
    Zt=[0]*n
    Bt=[0]*n
    
    v=[0]*n
    v[0]=initial_vol
    v_neg=[0]*n
    v_neg[0]=initial_vol
 
    S=[0]*n
    S[0]=initial_price
    S_neg=[0]*n #for antithetics
    S_neg[0]=initial_price
    
    #generate the volatility process
    for i in range(1,n):
        time[i]=time[i-1]+dt#update time
        
        #volatility process
        Zt[i]=Zt[i-1]+np.random.normal(0,np.sqrt(dt))
        dZ=Zt[i]-Zt[i-1]
        
        vol_drift=k*(theta-v[i-1])
        dv=vol_drift*dt + (volvol*np.sqrt(v[i-1])*dZ) + ((volvol*volvol)/4)*((dZ**2)-dt)
        v[i]=v[i-1]+dv
   
        vol_drift=k*(theta-v_neg[i-1])
        dv_neg=vol_drift*dt + (volvol*np.sqrt(v_neg[i-1])*-dZ) + ((volvol*volvol)/4)*(((-dZ)**2)-dt)
        v_neg[i]=v_neg[i-1]+dv_neg  
         
        #correlated brownian motion dWt and its antithetic
        Bt[i]=Bt[i-1]+np.random.normal(0,np.sqrt(dt))
        dB=Bt[i]-Bt[i-1]
        dW=p*dZ + np.sqrt(1-p**2)*dB
        dW_neg=p*-dZ + np.sqrt(1-p**2)*-dB
        
        #price generation
        dS=r*S[i-1]*dt + (np.sqrt(v[i-1])*S[i-1]*(dW))
        S[i]=S[i-1]+dS
        dS_neg=r*S_neg[i-1]*dt + (np.sqrt(v_neg[i-1])*S_neg[i-1]*(dW_neg))
        S_neg[i]=S_neg[i-1]+dS_neg
            
    return S[-1],S_neg[-1]   

In [9]:
#simulation for M, L times
payoff=np.zeros(M)
last_price=np.zeros(M)
last_price_antithetic=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #compute M paths
        last_price[i],last_price_antithetic[i]=Heston_antithetic(initial_price, initial_vol,r, k, theta, volvol ,T, p)

    #computing payoff
    payoff=np.exp(-r*T)*np.maximum(last_price-K,0)
    
    #computing antithetic payoff
    antithetic_payoff=np.exp(-r*T)*np.maximum(last_price_antithetic-K,0)
    
    #estimator
    estimator=(payoff+antithetic_payoff)/2
    mean_pay[x]=np.mean(estimator)
    
end = time.time()

print(end-start)

33.72781801223755


In [10]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

39.16588927956108
6.398367319357034


### Conditional

In [11]:
#function for computing Black Scholes price
def bs_price(vol, initial_price, r, T, K):
    d1=np.log(initial_price/K) + (r+(vol*vol)/2)*T
    d1=d1/(vol*np.sqrt(T))

    d2=d1- vol*np.sqrt(T)

    vanilla_price=initial_price*stats.norm.cdf(d1) - K*np.exp(-r*T)*stats.norm.cdf(d2)
    return vanilla_price

#volatility process and Zt
def vol_Zt(initial_vol,theta, volvol ,T):
    dt=T/n
    
    time=[0]*n
    v=[0]*n
    v[0]=initial_vol
    Zt=[0]*n
    dZ=[0]*n
    
    #generate the volatility process
    for i in range(1,n):
        time[i]=time[i-1]+dt
        
        Zt[i]=Zt[i-1]+np.random.normal(0,np.sqrt(dt))
        dZ[i-1]=Zt[i]-Zt[i-1]
        
        vol_drift=k*(theta-v[i-1])
        dv=vol_drift*dt + (volvol*np.sqrt(v[i-1])*dZ[i-1]) + ((volvol*volvol)/4)*((dZ[i-1]**2)-dt)
        v[i]=v[i-1]+dv
        
    return np.array(v),Zt,dZ

In [12]:
#simulation for M, L times
payoff=np.zeros(M)
last_price=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #v and brownian motion
        v, Zt, dZ=vol_Zt(initial_vol,theta, volvol, T)
        dt=T/n
        
        #Hull White formula for conditional expectation
        v0=v.sum()*dt
        v0=np.sqrt(v0/T)
        
        E=p*sum(np.sqrt(v)*dZ)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)

        
        #vol, initial_price, r, T, K
        payoff[i]=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K)
       
    mean_pay[x]=np.mean(payoff)
    
end = time.time()
print(end-start)

12.210964441299438


In [13]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

37.37649419936506
3.2113123805989274


### Antithetics + Conditional Method

In [14]:
import math
def Heston_CD_A(initial_vol, k, theta, volvol, T):
    n=1000
    dt=T/n
    time=[0]*n

    v=[0]*n
    v[0]=initial_vol
    Zt=[0]*n
    dZ=[0]*n
    
    v2=[0]*n
    v2[0]=initial_vol
    Zt2=[0]*n
    dZ2=[0]*n

    #generate volatility process and its antithetic 
    for i in range(1,n):
        time[i]=time[i-1]+dt
        
        Zt[i]=Zt[i-1]+np.random.normal(0,np.sqrt(dt))
        dZ[i-1]=Zt[i]-Zt[i-1]
        dZ2[i-1]=-dZ[i-1]
        Zt2[i]=Zt2[i-1]+dZ2[i-1]
        
        vol_drift=k*(theta-v[i-1])
        vol_drift2=k*(theta-v2[i-1])
        
        dv=vol_drift*dt + (volvol*np.sqrt(v[i-1])*dZ[i-1]) + ((volvol*volvol)/4)*((dZ[i-1]**2)-dt)
        dv2=vol_drift2*dt + (volvol*np.sqrt(v2[i-1])*dZ2[i-1]) + ((volvol*volvol)/4)*((dZ2[i-1]**2)-dt)
    
        v[i]=v[i-1]+dv  
        v2[i]=v2[i-1]+dv2            
    
    return np.array(v),Zt,dZ, np.array(v2),Zt2,dZ2

In [15]:
#simulation for M, L times
payoff=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #compute M paths
        v1,Zt1,dZ1,v2,Zt2,dZ2=Heston_CD_A(initial_vol, k, theta, volvol, T)
                
        #####1: compute first payoff
        v0=v1.sum()*dt
        v0=np.sqrt(v0/T)
        
        E=p*sum(np.sqrt(v1)*dZ1)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)
        
        #vol, initial_price, r, T, K
        pay1=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K)
    
        #####2 compute anithetic payoff
        v0=v2.sum()*dt
        v0=np.sqrt(v0/T)
        
        E=p*sum(np.sqrt(v2)*dZ2)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)
        
        #vol, initial_price, r, T, K
        pay2=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K) 
        
        #take the average
        payoff[i]=(pay1+pay2)/2
        
    mean_pay[x]=np.mean(payoff)

end = time.time()

print(end - start)

17.97016954421997


In [16]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

37.45943192467902
0.8524807341527247


### Control variate + Antithetics + Conditional

In [17]:
#simulation for M, L times
payoff=np.zeros(M)
last_vol=np.zeros(M)
mean_pay=np.zeros(L)

start = time.time()
for x in range(L):
    for i in range(M):
        #compute M paths
        v1,Zt1,dZ1,v2,Zt2,dZ2=Heston_CD_A(initial_vol, k, theta, volvol, T)
        
        #####1: first payoff
        v0=v1.sum()*dt
        v0=np.sqrt(v0/T)
        
        sqrt_v1=np.sqrt(v1)
        E=p*sum(sqrt_v1*dZ1)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)
        
        #vol, initial_price, r, T, K
        pay1=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K)
    
        #####2: antithetic payoff
        v0=v2.sum()*dt
        v0=np.sqrt(v0/T)
        
        sqrt_v2=np.sqrt(v2)
        E=p*sum(sqrt_v2*dZ2)-(((p**2)/2)*v0**2)*T
        S0=initial_price*np.exp(E)
        
        #vol, initial_price, r, T, K
        pay2=bs_price(v0*np.sqrt(1-p**2),S0,0,T, K) 
        
        #antithetic payoff
        payoff[i]=(pay1+pay2)/2
        
        #mean of last value of volatilities as control variate
        last_vol[i]=(v1[-1]**2+v2[-1]**2)/2

    #control variate estimator
    Y=payoff
    Z=last_vol
    c=-np.cov(Y,Z)[0][1]/np.var(Z)
  
    #computing new estimator (vector of M estimations)
    expected_v=(initial_vol*np.exp(-k*T))+theta*(1-np.exp(-k*T))
    var_v=(initial_vol*((volvol**2)/k)*(np.exp(-k*T)-np.exp(-2*k*T))) + (((theta*(volvol**2))/(2*k))*((1-np.exp(-k*T))**2))
    expected_v2=var_v+(expected_v**2)
    control_variate=Y+c*(Z-expected_v2)

    #now take mean of the M estimators to compute the Xth price
    mean_pay[x]=np.mean(control_variate)

end = time.time()

print(end - start)

17.617993593215942


In [18]:
print(np.mean(mean_pay))
print(np.std(mean_pay))

37.56961550391438
0.40752814153153605
